In [ ]:
import pandas as pd
import geojson

In [ ]:
from geojson import Feature, Point, FeatureCollection


houses_df = pd.read_csv('../data/curated/property/property_4_lat_lon.csv')  
supermarket_df = pd.read_csv('../data/raw/external/victoria_supermarkets.csv')  

houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
supermarket_df = supermarket_df.dropna(subset=['longitude', 'latitude'])


### create GeoJSON Feature
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),  
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

### convert the data into Geojson
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

supermarket_features = supermarket_df.apply(lambda row: create_geojson_feature(row, 'supermarket'), axis=1).tolist()
supermarket_feature_collection = FeatureCollection(supermarket_features)


### save the geo file
houses_geojson_output_path = '../data/raw/external/API/4/property_4_geojson.geojson'
supermarket_geojson_output_path = '../data/raw/external/API/4/victoria_supermarkets_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(supermarket_geojson_output_path, 'w') as f:
    geojson.dump(supermarket_feature_collection, f)


print("Geojson created")


In [ ]:
import geopandas as gpd
gdf = gpd.read_file('../data/raw/external/API/4/property_4_geojson.geojson')

num_features = len(gdf)

mid_index = num_features // 2

gdf1 = gdf.iloc[:mid_index]
gdf2 = gdf.iloc[mid_index:]

gdf1.to_file('../data/raw/external/API/4/property_4_1_geojson.geojson', driver='GeoJSON')
gdf2.to_file('../data/raw/external/API/4/property_4_2_geojson.geojson', driver='GeoJSON')

In [ ]:
import geojson
import openrouteservice
import pandas as pd
import time

### api client
api_key = '5b3ce3597851110001cf6248c2b2602de2dc43afa9f186cdfc39caca'  
client = openrouteservice.Client(key=api_key)

with open('../data/raw/external/API/4/property_4_1_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/4/victoria_supermarkets_geojson.geojson', 'r') as f:
    supermarket_geojson = geojson.load(f)

### create the function to get the latitude and longtitude
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
supermarket_coordinates = extract_coordinates(supermarket_geojson)

house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
supermarket_coords = [(lon, lat) for lat, lon, _ in supermarket_coordinates]



### using api to calculate the driving distance
def calculate_distance_matrix_in_batches(house_coords, supermarket_coords, batch_size=3):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + supermarket_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(supermarket_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
             ### find mini distance
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
    
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_supermarket_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_supermarket = supermarket_coordinates[closest_supermarket_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_supermarket': closest_supermarket,
                        'minimum_distance_supermarket': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # 防止触发速率限制
    
    return all_shortest_distances

### batch size using
shortest_distances = calculate_distance_matrix_in_batches(house_coords, supermarket_coords, batch_size=3)

### save
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/curated/external/API/4/closest_supermarket_distance_1_final_4.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved into {output_path}")


In [ ]:

### API key
api_key = '5b3ce3597851110001cf6248ef157f3775f94378ac76799c0f696a53'  
client = openrouteservice.Client(key=api_key)


with open('../data/raw/external/API/4/property_4_2_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/4/victoria_supermarkets_geojson.geojson', 'r') as f:
    supermarket_geojson = geojson.load(f)

### get latitude and longtitude
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
supermarket_coordinates = extract_coordinates(supermarket_geojson)

house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
supermarket_coords = [(lon, lat) for lat, lon, _ in supermarket_coordinates]



### start api calculating
def calculate_distance_matrix_in_batches(house_coords, supermarket_coords, batch_size=3):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + supermarket_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(supermarket_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            ### find mini distance
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_supermarket_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_supermarket = supermarket_coordinates[closest_supermarket_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_supermarket': closest_supermarket,
                        'minimum_distance_supermarket': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  
    
    return all_shortest_distances

### batch size using
shortest_distances = calculate_distance_matrix_in_batches(house_coords, supermarket_coords, batch_size=3)

### save
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/curated/external/API/4/closest_supermarket_distance_2_final_4.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"saved into {output_path}")

In [ ]:
### combine them
super1 = pd.read_csv('../data/curated/external/API/4/closest_supermarket_distance_1_final_4.csv')
super2 = pd.read_csv('../data/curated/external/API/4/closest_supermarket_distance_2_final_4.csv')
merged_df = pd.concat([super1, super2], ignore_index=True)
merged_df.to_csv('../data/curated/external/API/4/closest_supermarket_distance_final_4.csv', index=False)